In [1]:
import pickle
import re
import numpy as np
import sentencepiece as spm


In [2]:
%cd data/nagoya_corpus

C:\Users\yuuki\PycharmProjects\NLP\data\nagoya_corpus


In [3]:
data = []
for i in range(1, 130):
    with open('data{}.txt'.format(str(i).zfill(3)), encoding='utf-8') as f:
        data.extend(f.readlines())

## データ整形

In [4]:
person_pat = re.compile(r'[A-Z][0-9]+[：]')
unk_pat = re.compile(r'＊＊＊')
koyu_pat = re.compile(r'[＊].+[＊]')
kakko_pat = re.compile(r'[（][^（）]+[）]')
tagu_pat = re.compile(r'[＜][^＜＞]+[＞]')

data = [d.rstrip() for d in data if d[0] != '＠']
buf = []
for d in data:
    if not person_pat.match(d):
        buf[-1] += d
    else:
        buf.append(person_pat.sub('', d))
data = buf

# ＊＊＊を<unk>に置き換え
buf = []
for d in data:
    buf.append(unk_pat.sub('[UNK]', d))
data = buf

# ＊固有名詞＊を<unk>に置き換え
buf = []
for d in data:
    buf.append(koyu_pat.sub('[UNK]', d))
data = buf

# （・・・）を削除
buf = []
for d in data:
    buf.append(kakko_pat.sub('', d))
data = buf

# ＜・・・＞を削除
buf = []
for d in data:
    buf.append(tagu_pat.sub('', d))
data = buf

In [5]:
with open('data.txt', 'w', encoding='utf-8') as f:
    for d in data:
        f.write(d+'\n')

In [6]:
data = []
with open('data.txt', 'r', encoding='utf-8') as f:
    for line in f:
        data.append(line.rstrip())

In [7]:
data

['[UNK]の町というのはちいちゃくって、城壁がこう町全体をぐるっと回ってて、それが城壁の上を歩いても１時間ぐらいですよね。',
 '１時間かからないぐらいだね。４、５０分で。',
 'そうそう。ほいでさあ、ずっと歩いていたんだけど、そうすと上から、なんか町の中が見れるじゃん。あるよね。ほいでさあ、なんか途中でワンちゃんに会ったんだね。散歩をしてるワンちゃんに会ったんだ。',
 '城壁の上をやっぱ観光客なんだけどワンちゃん連れてきてる人たち結構多くて。',
 'で、こう、そのワンちゃんと２人を、なに、お父さんとお母さんと歩いて、ワンちゃんに会ったんだ。途中で。あワンちゃーんとか言ってなでて、ほいで、この人たちはこっち行って、あたしらこっち行ったじゃん。ずうーとこうやって回ってきてるの。また会っちゃって。ここで。そうしたら。',
 'おー、そら地球はやっぱり丸かったみたいだね。',
 'そうしたらそのワンちゃんがなんかか喜んじゃって、で、あたしの方に走ってきて、とびついてきちゃってさ。別にあたしさあ、別にさっきなでただけなのにさあ、なんかすごーいなつかれちゃってね。',
 'さっきね、別に、そんなになでてもいないんだよ。',
 'よしよしって言っただけなのに。',
 'あらワンちゃんだーとか言ってすれ違ったんだよ。普通に。それでその次のとき、向こうの方からはーっといってかけてくるじゃん。',
 'すごい勢いで走って。私、あ、あーさっきの犬だとか私たちが言っとるじゃん。あんで向こうの人たちも、あっ、さっき会った子たちねみたいな感じで気がついたじゃん。犬も気がついたじゃん。じゃははって走ってきちゃって、犬が。Ｘ：そうなんだ。',
 'ほいであちしなんかとびつかれちゃったよ。Ｘ：うそ。',
 '[UNK]って言ってさ。',
 'さっきちょっとなでただけなのにって。かわいかったね。',
 'そんでこれが２階建てのショッピングセンター。すごい古いんだけど、こん中に。',
 'えっショッピングセンターなんですか。これ。',
 'そう。[UNK]ぐらいたってるショッピングモールじゃんね。',
 'あ、モールなんだ。',
 'で、これ木でできた２階建てのとこにあって。',
 'で、中はモダンなんですか。それともレトロな感じで。',
 '中のお店は普通のショップなんだけど、床が。あの

In [8]:
spm.SentencePieceTrainer.Train('--control_symbols=<pad> --user_defined_symbols=[UNK] --input=data.txt --model_prefix=trained_model --vocab_size=8000 --add_dummy_prefix=false')

True

In [11]:
sp = spm.SentencePieceProcessor()

In [12]:
sp.Load('../../transformer/data/checkpoint/wiki-ja.model')

True

In [13]:
sp.IdToPiece(3)

'[PAD]'

In [14]:
token_data = [sp.EncodeAsIds(d) for d in data]

In [15]:
token_data.__len__()

72863

In [16]:
data.__len__()

72863

In [17]:
corpus = []
for t in token_data:
    corpus.append(t+[2])

In [18]:
[1,2] in decoder_inputs

NameError: name 'decoder_inputs' is not defined

In [19]:
encoder_inputs = [c for c in corpus][:-1]
decoder_inputs = [[1]+c for c in corpus][1:]
decoder_outputs = [c for c in corpus][1:]

In [20]:
generator_data = []
max_len = 30
for e_i, d_i, d_o in zip(encoder_inputs, decoder_inputs, decoder_outputs):
    if (e_i.__len__() <= max_len) and \
        (d_i.__len__() <= max_len) and \
        (d_o.__len__() <= max_len):
        if [2] != e_i and \
            [1, 2] != d_i and \
            [2] != d_o:
            generator_data.append([e_i, d_i, d_o])

In [21]:
list(map(lambda x:sp.decode_ids(x), np.array(generator_data)[:, 0].tolist()))

['さっきね、別に、そんなになでてもいないんだよ。',
 'ほいであちしなんかとびつかれちゃったよ。 ⁇ :うそ。',
 '[ ⁇ ]って言ってさ。',
 'さっきちょっとなでただけなのにって。かわいかったね。',
 'そんでこれが2階建てのショッピングセンター。すごい古いんだけど、こん中に。',
 'えっショッピングセンターなんですか。これ。',
 'そう。[ ⁇ ]ぐらいたってるショッピングモールじゃんね。',
 'あ、モールなんだ。',
 'で、これ木でできた2階建てのとこにあって。',
 'で、中はモダンなんですか。それともレトロな感じで。',
 '中のお店は普通のショップなんだけど、床が。あのー',
 '木?',
 '木で、フロアとかが木で、で、このところの柱も木で。だから',
 '普通にこう女性服が売ってたりとか。',
 '別にいろんな。',
 'ジーンズの店もあったとか、小物のお店もあったり。',
 'そう。いろんな[ ⁇ ]。',
 '靴、売ってたりとか。',
 'だから、がい、そとみだけは。なんか',
 '昔ながらの。',
 'そ、それを守って。中は。',
 '普通になってる。近代的になってる。',
 'ほんでなんか、昔の遺跡が出てきちゃって、ショッピングモール造ろうとしたら。',
 'うん。',
 'でまた通り過ぎちゃって、ないよねっとか言って。',
 'ほいでさ、あたしが、あっ、あそこにあるとか言ってさ、見つけたんだよね。',
 'ここの2階。',
 '言って、見にいったら、なんか。',
 'トイミュージアムって書いてあったの。',
 'クローズド。',
 'クローズド。',
 '終わってるやん。',
 'それでガラスの中。',
 'またぷりぷりおこって、私が何でーとか言って、私このために来た。',
 'テレフォンカードに続く第2段。',
 'でも、すてきでしょう。こここう2階もこうあって。',
 '下もショップなんだけど。',
 'ここの中の本屋さんが70パー引きだった。',
 'そうそう。70パー引き。',
 '70パー引き?本が?',
 'ほいでねー。',
 'それがたぶん1000円ぐらいだったね。4ポンドいくら。',
 'えーとね3ポンド99だもん。',
 'あーだもんで1000円しない。',
 'これは買うっきゃないねとかって言って。

In [22]:
list(map(lambda x:sp.decode_ids(x), np.array(generator_data)[:, 1].tolist()))

['よしよしって言っただけなのに。',
 '[ ⁇ ]って言ってさ。',
 'さっきちょっとなでただけなのにって。かわいかったね。',
 'そんでこれが2階建てのショッピングセンター。すごい古いんだけど、こん中に。',
 'えっショッピングセンターなんですか。これ。',
 'そう。[ ⁇ ]ぐらいたってるショッピングモールじゃんね。',
 'あ、モールなんだ。',
 'で、これ木でできた2階建てのとこにあって。',
 'で、中はモダンなんですか。それともレトロな感じで。',
 '中のお店は普通のショップなんだけど、床が。あのー',
 '木?',
 '木で、フロアとかが木で、で、このところの柱も木で。だから',
 '普通にこう女性服が売ってたりとか。',
 '別にいろんな。',
 'ジーンズの店もあったとか、小物のお店もあったり。',
 'そう。いろんな[ ⁇ ]。',
 '靴、売ってたりとか。',
 'だから、がい、そとみだけは。なんか',
 '昔ながらの。',
 'そ、それを守って。中は。',
 '普通になってる。近代的になってる。',
 'ほんでなんか、昔の遺跡が出てきちゃって、ショッピングモール造ろうとしたら。',
 '壊せなくなったの?',
 'でまた通り過ぎちゃって、ないよねっとか言って。',
 'ほいでさ、あたしが、あっ、あそこにあるとか言ってさ、見つけたんだよね。',
 'それが。',
 '言って、見にいったら、なんか。',
 'トイミュージアムって書いてあったの。',
 'クローズド。',
 'クローズド。',
 '終わってるやん。',
 'それでガラスの中。',
 'またぷりぷりおこって、私が何でーとか言って、私このために来た。',
 'テレフォンカードに続く第2段。',
 'でも、すてきでしょう。こここう2階もこうあって。',
 '下もショップなんだけど。',
 'ここの中の本屋さんが70パー引きだった。',
 'そうそう。70パー引き。',
 '70パー引き?本が?',
 'ほいでねー。',
 'そいで買ったんだ。',
 'えーとね3ポンド99だもん。',
 'あーだもんで1000円しない。',
 'これは買うっきゃないねとかって言って。',
 '安いね。',
 'そうそう、すごい。それを。',
 'なんで閉店セールとかではなくて。',
 'じ

In [23]:
buf = []
for g in generator_data:
    buf.append([
        g[0] + [3] * (30 - g[0].__len__()),
        g[1] + [3] * (30 - g[1].__len__()),
        g[2] + [3] * (30 - g[2].__len__()),
    ])

In [24]:
buf.__len__()

51374

In [25]:
generator_data = np.array(buf)

In [98]:
generator_data[:, 2, :, None]

array([[[1645],
        [1645],
        [ 903],
        ...,
        [   3],
        [   3],
        [   3]],

       [[ 892],
        [2605],
        [ 141],
        ...,
        [   3],
        [   3],
        [   3]],

       [[   4],
        [ 244],
        [  31],
        ...,
        [   3],
        [   3],
        [   3]],

       ...,

       [[  88],
        [   6],
        [   2],
        ...,
        [   3],
        [   3],
        [   3]],

       [[  23],
        [   5],
        [1552],
        ...,
        [   3],
        [   3],
        [   3]],

       [[5000],
        [ 199],
        [4022],
        ...,
        [   3],
        [   3],
        [   3]]])

In [26]:
with open('generator_data.pickle', 'wb') as f:
    pickle.dump(generator_data, f)

In [78]:
generator_data[:, 0].shape

(57580, 30)

In [81]:
generator_data[1:129, 1].shape

(128, 30)

In [28]:
sp.decode_ids([1,2,3,4,5,100, 101])

'ドり'